In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

E:\New volume\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1 )


In [7]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]

In [8]:
dep=dataset["Purchased"]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(indep,dep,test_size=1/3, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid ={'criterion':['gini','entropy'],
              'max_features':['auto','sqrt','log2'],
               'n_estimators':[10,100]}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train, y_train)




Fitting 5 folds for each of 12 candidates, totalling 60 fits


E:\New volume\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "E:\New volume\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "E:\New volume\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "E:\New volume\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "E:\New volume\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in valida

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

In [14]:
grid_predictions=grid.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [16]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions, average='weighted')

print("The f1_macro value for best parameter{}:",format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{}: {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100} 0.9256751954513147


In [18]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[79  6]
 [ 4 45]]


In [19]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9606242496998799